In [35]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import tensorflow as tf
import tensorflow_probability as tfp
# from tensorflow_probability.layers import DenseVariational
from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization
import sys
sys.path.append('../..')
from modules import utils
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

#### The dataset

In [19]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [20]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [21]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)

final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


In [22]:
idx=1
device_indices = final_df[final_df.latitude==latitudes[idx]].index
test_dataset = final_df.loc[device_indices]
train_dataset = pd.concat([final_df, test_dataset]).drop_duplicates(keep=False)
len(final_df), len(train_dataset), len(test_dataset)

(13653, 12801, 852)

#### The wine dataset

In [23]:
# import tensorflow_datasets as tfds

In [24]:
# def get_train_test_splits(train_size, batch_size=1):
#     dataset = (
#         tfds.load(name='wine_quality', as_supervised=True, split='train')
#         .map(lambda x, y: (x, tf.cast(y, tf.float32)))
#         .prefetch(buffer_size=dataset_size)
#         .cache()
        
#     )
#     print(f'type of dataset: {type(dataset)}')
# #     print(f'shape of dataset: {dataset._input_dataset._input_dataset._batch_size.numpy()}')
    
#     train_dataset= (dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size))
#     test_dataset = dataset.skip(train_size).batch(batch_size)
#     return train_dataset, test_dataset

In [25]:
# dataset_size = 4898
# batch_size = 256
# train_size = int(dataset_size*0.85)
# train_dataset, test_dataset = get_train_test_splits(train_size, batch_size)

In [26]:
# type(train_dataset)

#### Defining the model

In [27]:
hidden_units = [8, 8]
learning_rate = 0.001

In [66]:
def run_experiment(model, loss, train_dataset, test_dataset):
    X_train = train_dataset.iloc[:, 0:-1]
    y_train = train_dataset.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1, 1)
    
    X_test = test_dataset.iloc[:, 0:-1]
    y_test = test_dataset.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1, 1)
    
    model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate), loss=loss, 
                  metrics = [keras.metrics.RootMeanSquaredError()])
    print('Start training the model ...')
    model.fit(X_train, y_train, epochs=num_epochs, validation_split=0.2)
    print('Model training finished.')
    print('Evaluating model performance ...')
    
#     y_pred = model.predict(X_test) 
    y_pred = model.predict(X_test).flatten()
    rmse_a = sqrt(mean_squared_error(y_test, y_pred))
    _, rmse_b = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test RMSEs: {round(rmse_a, 3)}, {round(rmse_b, 3)}')

In [67]:
FEATURE_NAMES = ['time', 'latitude', 'longitude']

In [68]:
# def create_model_inputs():
#     inputs = {}
#     for feature_name in FEATURE_NAMES:
#         inputs[feature_name] = Input(name=feature_name, shape=(1,), dtype=tf.float32)
#     return inputs

In [69]:
def create_baseline_model(input_size, dropout=0.1):
#     inputs = create_model_inputs()
#     input_values = [value for _,value in sorted(inputs.items())]
#     features = layers.concatenate(input_values)
#     features = BatchNormalization()(features) #to remove
    
#     for units in hidden_units:
#         features = Dense(units, activation='sigmoid')(features)
#     outputs = Dense(units=1)(features)
#     model = keras.Model(inputs=inputs, outputs=outputs)
    
    model = tf.keras.Sequential() 
    model.add(Input(shape=(input_size,), name='Input-Layer')) 
    model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 
    
    return model

In [71]:
# dataset_size = len(final_df)
# batch_size=256
# num_epochs=500
# mse_loss = keras.losses.MeanSquaredError()
# baseline_model = create_baseline_model(final_df.shape[1]-1)
# run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

#### Bayesian NN

In [ ]:
def bnn_not_used_chatgpt_i_think(X_train, y_train):
    model = tf.keras.Sequential([
    tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn),
    tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn),
    tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn),
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
    model.fit(x_train, y_train, epochs=10, batch_size=32)
    
    return model

In [72]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size +bias_size
    prior_model = keras.Sequential(
        [tfp.layers.DistributionLambda(
            lambda t:tfp.distributions.MultivariateNormalDiag(
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            ))
        ])
    return prior_model

In [73]:
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size +bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
         tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [74]:
def create_bnn_model(input_size, train_size, dropout=0.1): #train_size = len(X_train)
#     inputs = create_model_inputs()
#     features = keras.layers.concatenate(list(inputs.values()))
#     features = layers.BatchNormalization()(features)


    # Create hidden layers with weight uncertainty using the DenseVariational layer.
#     for units in hidden_units:
#         features = tfp.layers.DenseVariational(
#             units=units,
#             make_prior_fn=prior,
#             make_posterior_fn=posterior,
#             kl_weight=1 / train_size,
#             activation='sigmoid',
#         )(features)
# The output is deterministic: a single point estimate.
#     outputs = layers.Dense(units=1)(features)
#     model = keras.Model(inputs=inputs, outputs=outputs)
    
    model = tf.keras.Sequential() 
    model.add(Input(shape=(input_size,), name='Input-Layer')) 
    model.add(Dropout(rate=dropout))
    model.add(tfp.layers.DenseVariational(units=128, make_prior_fn=prior, make_posterior_fn=posterior, 
              activation='relu', name='Hidden-Layer-1', kl_weight=1/train_size))
    model.add(Dropout(rate=dropout))
    model.add(tfp.layers.DenseVariational(units=32, make_prior_fn=prior, make_posterior_fn=posterior, 
              activation='relu', name='Hidden-Layer-2', kl_weight=1/train_size))
    model.add(Dropout(rate=dropout))
    model.add(Dense(1))
    
    return model

In [76]:
# num_epochs = 500
# train_sample_size = 100
# small_train_dataset = train_dataset[:100]

# bnn_model_small = create_bnn_model(final_df.shape[1]-1, train_sample_size)
# run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

In [ ]:
num_epochs = 500
bnn_model_full = create_bnn_model(final_df.shape[1]-1, len(train_dataset))
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)
# compute_predictions(bnn_model_full)

Start training the model ...
Epoch 1/500
320/320 [==============================] - 50s 153ms/step - loss: 31576167546880.0000 - root_mean_squared_error: 5619267.5000 - val_loss: 23308875595776.0000 - val_root_mean_squared_error: 4827926.5000
Epoch 2/500
320/320 [==============================] - 50s 155ms/step - loss: 16744032763904.0000 - root_mean_squared_error: 4091947.2500 - val_loss: 4432363782144.0000 - val_root_mean_squared_error: 2105318.0000
Epoch 3/500
320/320 [==============================] - 74s 231ms/step - loss: 2799588343808.0000 - root_mean_squared_error: 1673197.0000 - val_loss: 554054844416.0000 - val_root_mean_squared_error: 744348.6250
Epoch 4/500
320/320 [==============================] - 46s 142ms/step - loss: 249280364544.0000 - root_mean_squared_error: 499279.8438 - val_loss: 24330100736.0000 - val_root_mean_squared_error: 155981.0938
Epoch 5/500
320/320 [==============================] - 57s 179ms/step - loss: 6759941120.0000 - root_mean_squared_error: 82218.

320/320 [==============================] - 47s 148ms/step - loss: 145909.4062 - root_mean_squared_error: 381.9796 - val_loss: 10636.9082 - val_root_mean_squared_error: 103.1302
Epoch 43/500
320/320 [==============================] - 48s 148ms/step - loss: 126796.6484 - root_mean_squared_error: 356.0836 - val_loss: 1471.8778 - val_root_mean_squared_error: 38.3507
Epoch 44/500
320/320 [==============================] - 48s 149ms/step - loss: 177367.6250 - root_mean_squared_error: 421.1491 - val_loss: 1467.7167 - val_root_mean_squared_error: 38.2960
Epoch 45/500
320/320 [==============================] - 47s 148ms/step - loss: 153645.5625 - root_mean_squared_error: 391.9750 - val_loss: 63173.1719 - val_root_mean_squared_error: 251.3404
Epoch 46/500
320/320 [==============================] - 47s 148ms/step - loss: 142344.8594 - root_mean_squared_error: 377.2847 - val_loss: 1461.6820 - val_root_mean_squared_error: 38.2164
Epoch 47/500
320/320 [==============================] - 47s 148ms/ste

320/320 [==============================] - 47s 145ms/step - loss: 112445.9766 - root_mean_squared_error: 335.3266 - val_loss: 5426.0127 - val_root_mean_squared_error: 73.6474
Epoch 86/500
320/320 [==============================] - 47s 146ms/step - loss: 82858.2188 - root_mean_squared_error: 287.8474 - val_loss: 1307.1439 - val_root_mean_squared_error: 36.1257
Epoch 87/500
320/320 [==============================] - 47s 145ms/step - loss: 122230.8906 - root_mean_squared_error: 349.6124 - val_loss: 1498.8579 - val_root_mean_squared_error: 38.6878
Epoch 88/500
320/320 [==============================] - 47s 145ms/step - loss: 72422.0859 - root_mean_squared_error: 269.1096 - val_loss: 1444.8372 - val_root_mean_squared_error: 37.9830
Epoch 89/500
320/320 [==============================] - 47s 145ms/step - loss: 246264.1562 - root_mean_squared_error: 496.2478 - val_loss: 5271.7280 - val_root_mean_squared_error: 72.5920
Epoch 90/500
320/320 [==============================] - 47s 146ms/step - lo

320/320 [==============================] - 49s 153ms/step - loss: 516503.9375 - root_mean_squared_error: 718.6802 - val_loss: 7299.4126 - val_root_mean_squared_error: 85.4228
Epoch 129/500
320/320 [==============================] - 50s 156ms/step - loss: 619671.3125 - root_mean_squared_error: 787.1905 - val_loss: 1171.5105 - val_root_mean_squared_error: 34.1933
Epoch 130/500
320/320 [==============================] - 49s 152ms/step - loss: 449120.1562 - root_mean_squared_error: 670.1625 - val_loss: 1169.7444 - val_root_mean_squared_error: 34.1675
Epoch 131/500
320/320 [==============================] - 53s 166ms/step - loss: 155558.8750 - root_mean_squared_error: 394.4064 - val_loss: 1166.1562 - val_root_mean_squared_error: 34.1146
Epoch 132/500
320/320 [==============================] - 51s 160ms/step - loss: 908780.8750 - root_mean_squared_error: 953.2989 - val_loss: 1165.3314 - val_root_mean_squared_error: 34.1030
Epoch 133/500
320/320 [==============================] - 50s 157ms/st

320/320 [==============================] - 48s 151ms/step - loss: 418071.5625 - root_mean_squared_error: 646.5831 - val_loss: 1062.5685 - val_root_mean_squared_error: 32.5666
Epoch 172/500
320/320 [==============================] - 48s 151ms/step - loss: 216552.5000 - root_mean_squared_error: 465.3500 - val_loss: 1873.8612 - val_root_mean_squared_error: 43.2654
Epoch 173/500
320/320 [==============================] - 47s 148ms/step - loss: 417229.0625 - root_mean_squared_error: 645.9313 - val_loss: 1055.8293 - val_root_mean_squared_error: 32.4633
Epoch 174/500
320/320 [==============================] - 48s 150ms/step - loss: 305171.7812 - root_mean_squared_error: 552.4218 - val_loss: 1052.9420 - val_root_mean_squared_error: 32.4190
Epoch 175/500
320/320 [==============================] - 49s 154ms/step - loss: 1591029.7500 - root_mean_squared_error: 1261.3596 - val_loss: 1052.2419 - val_root_mean_squared_error: 32.4086
Epoch 176/500
320/320 [==============================] - 48s 151ms/

320/320 [==============================] - 87s 273ms/step - loss: 420416.1875 - root_mean_squared_error: 648.3940 - val_loss: 952.6741 - val_root_mean_squared_error: 30.8420
Epoch 215/500
320/320 [==============================] - 91s 285ms/step - loss: 515414.1562 - root_mean_squared_error: 717.9228 - val_loss: 953.2686 - val_root_mean_squared_error: 30.8519
Epoch 216/500
320/320 [==============================] - 92s 288ms/step - loss: 166895.6406 - root_mean_squared_error: 408.5269 - val_loss: 949.7714 - val_root_mean_squared_error: 30.7953
Epoch 217/500
320/320 [==============================] - 91s 283ms/step - loss: 215577.2500 - root_mean_squared_error: 464.3015 - val_loss: 947.7396 - val_root_mean_squared_error: 30.7626
Epoch 218/500
320/320 [==============================] - 95s 298ms/step - loss: 50375.9180 - root_mean_squared_error: 224.4427 - val_loss: 943.8883 - val_root_mean_squared_error: 30.7002
Epoch 219/500
320/320 [==============================] - 104s 325ms/step - 

In [65]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f'Predictions mean: {round(prediction_mean[idx], 2)}, '
            f'min: {round(prediction_min[idx], 2)}, '
            f'max: {round(prediction_max[idx], 2)}, '
            f'range: {round(prediction_range[idx], 2)} - '
            f'Actual: {targets[idx]}'
        )


compute_predictions(bnn_model_small)

NameError: name 'examples' is not defined

#### Define the model

In [ ]:
def normal_exp(params):
    return tfd.Normal(loc=params[:, 0:1], scale=tf.math.exp(params[:, 1:2]))

In [ ]:
inputs = Input(shape=(1,))
hidden = Dense(200, activation='relu')(inputs)
hidden = Dropout(0.1)(hidden, training=True)
hidden = Dense(500, activation='relu')(hidden)
hidden = Dropout(0.1)(hidden, training=True)
hidden = Dense(500, activation='relu')(hidden)
hideen = Dropout(0.1)(hidden, training=True)
hidden = Dense(500, activation='relu')(hidden)
hidden = Dropout(0.1)(hidden, training=True)
hidden = Dense(200, activation='relu')(hidden)
hidden = Dropout(0.1)(hidden, training=True)
params_mc = Dense(2)(hidden)
dist_mc = tfp.layers.DistributionLambda(normal_exp, name='normal_exp')(params_mc)

model_mc = Model(inputs=inputs, outputs=dist_mc)
model_mc.compile(Adam(learning_rate=0.0002, loss=NLL))
model_mc.summary()